In [ ]:
!pip install datasets huggingface_hub -q

In [ ]:
# Use a pipeline as a high-level helper
from google.colab import drive





In [ ]:
import pandas as pd
drive.mount('/content/drive')
# Ruta al archivo
ruta_archivo = "/content/drive/MyDrive/Colab Notebooks/reglasmentoReducido.txt"
#ruta_archivo="/content/PreguntasRespuestasGPT.txt"

# Leer el archivo de texto del reglamento en español
with open(ruta_archivo, "r", encoding="utf-8") as file:
    texto = file.read()

# Divide el texto en artículos usando "Documento:" como delimitador
articulos = texto.split("Documento:")

# Eliminar posibles espacios en blanco o líneas vacías y reconstruir cada artículo con "Documento:" al principio
articulos = ["Documento:" + articulo.strip() for articulo in articulos if articulo.strip()]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Cantidad de articulos en la lista
print(f"Se encontraron {len(articulos)} artículos.")

# Imprimir los primeros 3 artículos como ejemplo
for articulo in articulos[:3]:
    print(articulo)
    print("-"*80)

Se encontraron 9 artículos.
Documento:8
Articulo: 3
Capitulo: I - CONVOCATORIA

Norma General: Las convocatorias para la designación por Concurso de Profesores Ordinarios titulares, asociados y adjuntos para las diferentes cátedras, áreas, núcleos, asignaturas que integran los núcleos, disciplinas, laboratorio de las Facultades que integran esta Universidad se regirán por las disposiciones del presente Reglamento y las que en consecuencia dicten las Unidades Académicas, que deberán ser aprobadas por el Consejo Superior.
--------------------------------------------------------------------------------
Documento:8
Articulo: 4
Capitulo: I - CONVOCATORIA

Cada Unidad Académica mediante Resolución del Consejo Directivo propondrá al Consejo Superior la provisión de cargos de profesores por concurso especificando lo siguiente: a) las cátedras, áreas, núcleos, asignaturas que integran los núcleos, disciplinas, laboratorio a concursar. b) La categoría. c) La dedicación. d) Si la imputación presu

⛳ **En esta parte que sigue suponemos que encontramos la forma de que un LLM nos devuelva preguntas y respuestas para cada articulo**  (El posible output solamente es para ejemplificar, no esta hecho con el documento)

In [ ]:
PosibleOutput1 = "Pregunta: ¿Cómo deben presentar las Unidades Académicas la solicitud de provisión de cargos de profesores? Respuesta: Las Unidades Académicas deben presentar la solicitud mediante Resolución del Consejo Directivo. Pregunta: ¿Qué información deben incluir las Unidades Académicas en la solicitud de provisión de cargos de profesores? Respuesta: Las Unidades Académicas deben incluir la siguiente información: las cátedras, áreas, núcleos, asignaturas que integran los núcleos, disciplinas, laboratorio a concursar, la categoría y la dedicación. Pregunta: ¿Qué partida presupuestaria está relacionada con el cargo llamado? Respuesta: La partida presupuestaria relacionada con el cargo llamado es la partida Gastos de Personal. Pregunta: ¿Cuánto tiempo tiene el Consejo Superior para resolver sobre la solicitud de llamado? Respuesta: El Consejo Superior tiene treinta (30) días para resolver sobre la solicitud de llamado."

In [ ]:
PosibleOutput2 = "Pregunta: ¿Cuáles son los pasos para solicitar un permiso de investigación en el campus? Respuesta: Los pasos para solicitar un permiso de investigación en el campus incluyen llenar el formulario de solicitud, obtener la aprobación del supervisor del departamento, y presentar el formulario a la oficina de administración. Pregunta: ¿Qué documentos son necesarios para inscribirse en un curso de verano? Respuesta: Los documentos necesarios para inscribirse en un curso de verano son una copia del DNI, el formulario de inscripción completado, y el comprobante de pago de la matrícula. Pregunta: ¿Cuáles son los criterios para recibir una beca de excelencia académica? Respuesta: Los criterios para recibir una beca de excelencia académica incluyen tener un promedio general de al menos 8.5, no tener materias reprobadas, y presentar una carta de recomendación de un profesor. Pregunta: ¿Qué servicios ofrece la biblioteca para los estudiantes de posgrado? Respuesta: La biblioteca ofrece servicio"


In [ ]:
list_QA = [PosibleOutput1, PosibleOutput2]

In [ ]:
def generar_pregunta_respuesta(articulo):
    # Procesar múltiples preguntas y respuestas
    pairs = []
    segments = articulo.split("Pregunta:")
    for segment in segments[1:]:  # Ignorar el primer split que no contiene pregunta
        if "Respuesta:" in segment:
            question = segment.split("Respuesta:")[0].strip()
            answer = segment.split("Respuesta:")[1].strip()
            pairs.append({"from": "human", "value": question})
            pairs.append({"from": "gpt", "value": answer})
    return pairs

In [ ]:
def generar_dataset(lista_articulos):
    dataset = []
    for articulo in lista_articulos:
        pairs = generar_pregunta_respuesta(articulo)
        dataset.append({'messages': pairs})
    return pd.DataFrame(dataset)

In [ ]:
dataframe = generar_dataset(list_QA)


In [ ]:
print(dataframe)
# Guardar el DataFrame en un archivo CSV si fuera necesario
#df.to_csv('dataset.csv', index=False)



                                            messages
0  [{'from': 'human', 'value': '¿Cómo deben prese...
1  [{'from': 'human', 'value': '¿Cuáles son los p...


La parte de acá sería para subir el dataset a Hugging Face

In [ ]:

from datasets import Dataset, DatasetDict


In [ ]:
# Convierte el DataFrame en un Dataset de Hugging Face
dataset = Dataset.from_pandas(dataframe)

# Crea un diccionario de datasets si hay más splits (No creo que sea necesario por ahora)
#dataset_dict = DatasetDict({"train": dataset})

# Sube el dataset a Hugging Face
#dataset_dict.push_to_hub("tu_nombre_de_usuario/tu_nombre_de_dataset")